In [1]:
import os
from collections import OrderedDict

In [2]:
from varsim import BasePopulation, BaseSimulation, BaseModel, BasicSimulation

In [3]:
import varsim

In [4]:
import numpy as np
import pandas as pd

In [5]:
class StupidPopulation(BasePopulation):
    def __init__(self):
        self.t0 = [59580.03, 59580.05]
        self.f = [0.002, 0.004]
        self.minvals = [-0.2, -0.5]
        self.maxvals = [0.5, 1.0]
        self._paramsTable = None
    
    @property
    def rng_model(self):
        return None
    
    @property
    def paramsTable(self):
        if self._paramsTable is None:
            df = pd.DataFrame(dict(t0=self.t0, f=self.f, min=self.minvals,
                                   max=self.maxvals, idx=self.idxvalues))
            self._paramsTable = df.set_index('idx')
        return self._paramsTable
    
    def modelParams(self, idx):
        return OrderedDict(self.paramsTable.loc[idx])
    @property
    def idxvalues(self):
        return list(x for x in (0, 1))
    @property
    def numSources(self):
        return sum(1 for i in self.idxvalues)

In [6]:
sp = StupidPopulation()

In [7]:
sp.paramsTable

,f,max,min,t0
idx,,,,
0,0.002,0.5,-0.2,59580.03
1,0.004,1.0,-0.5,59580.05


In [8]:
class StupidModel(BaseModel):
    def __init__(self):
        self.t0 = None
        self.f = None
        self._maxday = None
        self._minday = None
        self.z = None
        
    @property 
    def modelParameters(self):
        return dict(f=self.f, maxday=self._maxday, minday=self._minday, x=self.z)

    def setModelParameters(self, params):
        self.f = params['f']
        self.t0 = params['t0']
        self._maxday = params['max'] 
        self._minday = params['min']
        return
    @property
    def minMjd(self):
        return self._minMjd
    
    @property
    def maxMjd(self):
        return self._maxday + self.t0
    
    @property
    def minMjd(self):
        return self._minday + self.t0
    
    def modelFlux(self, mjd, bandpassobj):
        #d = mjd[(mjd > self.minMjd) & (mjd < self.maxMjd)]
        vals = np.sin((mjd - self.t0)/ self.f)
        return vals

In [9]:
model = StupidModel()

In [10]:
from lsst.sims.photUtils import BandpassDict
from lsst.sims.catUtils.supernovae import SNObject
from sndata import LightCurve

In [11]:
class StupidSimulation(BasicSimulation):
    def __init__(self, rng, rng_obs, pointings, population, model,
                 bandpassDict='lsst', timeRange=(-70., 100.)):
        self._rng = rng
        self._rng_obs = rng_obs
        self._pointings = pointings
        self.model = model
        self.population = population
        self.timeRange = timeRange
        if bandpassDict == 'lsst':
            self.bandPasses = BandpassDict.loadTotalBandpassesFromFiles()
        else:
            self.bandPasses = bandpassDict
        
    @property
    def randomState(self):
        return self._rng
    @property
    def randomObs(self):
        return self._rng_obs
    
    @property
    def pointings(self):
        return self._pointings

In [12]:
import sndata

In [13]:
# The set of pointings randomly kept for convenience.
example_data = varsim.example_data
pointings_File = os.path.join(example_data, 'example_pointings.csv')
pointings = pd.read_csv(pointings_File, index_col='obsHistID')

In [14]:
ss =  StupidSimulation(rng=np.random.RandomState(0), rng_obs=np.random.RandomState(10), 
                       pointings=pointings, population=sp, model=model,
                       bandpassDict='lsst', timeRange=(59570., 59590))

In [15]:
ss.population.paramsTable

,f,max,min,t0
idx,,,,
0,0.002,0.5,-0.2,59580.03
1,0.004,1.0,-0.5,59580.05


In [16]:
ss.pointings.head()

,sessionID,propID,fieldID,fieldRA,fieldDec,filter,expDate,expMJD,night,visitTime,...,moonBright,darkBright,rawSeeing,wind,humidity,slewDist,slewTime,fiveSigmaDepth,ditheredRA,ditheredDec
obsHistID,,,,,,,,,,,,,,,,,,,,,
1,1016,54,316,1.676483,-1.082473,y,2922,59580.033829,0,34.0,...,0.0,105.610378,0.920473,0.0,0.0,1.620307,0.000000,21.021236,1.643930,-1.108924
14,1016,54,1118,1.709686,-0.607725,y,3424,59580.039637,0,34.0,...,0.0,103.104582,0.903960,0.0,0.0,0.055856,4.599568,21.067249,1.691083,-0.634176
13,1016,54,1220,1.664386,-0.566519,y,3386,59580.039190,0,34.0,...,0.0,101.147431,0.903960,0.0,0.0,0.048200,4.776511,21.091149,1.646286,-0.592970
12,1016,54,1212,1.608166,-0.575819,y,3347,59580.038741,0,34.0,...,0.0,98.109397,0.903960,0.0,0.0,0.055665,4.739998,21.128319,1.589958,-0.602270
11,1016,54,1110,1.651280,-0.618574,y,3308,59580.038293,0,34.0,...,0.0,100.125895,0.903960,0.0,0.0,0.056173,4.499824,21.103635,1.632535,-0.645025


In [17]:
ss.lc(0).lightCurve

,mjd,fluxerr,deviations,flux,fieldID,zpsys,band,fiveSigmaDepth,obsHistID,objID,ModelFlux,zp
expID,,,,,,,,,,,,
0_1,59580.033829,8.527999e-07,1.764052,0.941515,316,ab,lssty,21.021236,1,0,0.941513,0.0
0_14,59580.039637,7.464606e-10,0.400157,-0.994376,1118,ab,lssty,21.067249,14,0,-0.994376,0.0
0_13,59580.039190,7.301663e-10,0.978738,-0.993118,1220,ab,lssty,21.091149,13,0,-0.993118,0.0
0_12,59580.038741,7.055271e-10,2.240893,-0.942123,1212,ab,lssty,21.128319,12,0,-0.942123,0.0
0_11,59580.038293,7.217953e-10,1.867558,-0.844112,1110,ab,lssty,21.103635,11,0,-0.844112,0.0
0_10,59580.037847,7.537305e-10,-0.977278,-0.704634,1021,ab,lssty,21.056753,10,0,-0.704634,0.0
0_9,59580.037401,7.814629e-10,0.950088,-0.530260,957,ab,lssty,21.017621,9,0,-0.530260,0.0
0_8,59580.036955,7.766471e-10,-0.151357,-0.329626,857,ab,lssty,21.024316,8,0,-0.329626,0.0
0_7,59580.036509,7.732405e-10,-0.103219,-0.112668,770,ab,lssty,21.029077,7,0,-0.112668,0.0


In [18]:
ss.lc(0, add_cols=('fieldID', 'objID', 'obsHistID')).lightCurve

,mjd,fluxerr,zpsys,obsHistID,objID,band,flux,fieldID,zp
expID,,,,,,,,,
0_1,59580.033829,8.527999e-07,ab,1,0,lssty,0.941513,316,0.0
0_14,59580.039637,7.464606e-10,ab,14,0,lssty,-0.994376,1118,0.0
0_13,59580.039190,7.301663e-10,ab,13,0,lssty,-0.993118,1220,0.0
0_12,59580.038741,7.055271e-10,ab,12,0,lssty,-0.942123,1212,0.0
0_11,59580.038293,7.217953e-10,ab,11,0,lssty,-0.844112,1110,0.0
0_10,59580.037847,7.537305e-10,ab,10,0,lssty,-0.704634,1021,0.0
0_9,59580.037401,7.814629e-10,ab,9,0,lssty,-0.530260,957,0.0
0_8,59580.036955,7.766471e-10,ab,8,0,lssty,-0.329626,857,0.0
0_7,59580.036509,7.732405e-10,ab,7,0,lssty,-0.112668,770,0.0


In [22]:
!rm test.hdf
ss.write_lc(1, 'test.hdf')

/Users/rbiswas/soft/LSST/DarwinX86/miniconda2/4.2.12.lsst1/lib/python2.7/site-packages/tables/path.py:112: NaturalNameWarning: object name is not a valid Python identifier: '0'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)


In [23]:
!rm phot.hdf
!rm pop.hdf
ss.write_simulation('pop.hdf', 'phot.hdf')

rm: phot.hdf: No such file or directory
rm: pop.hdf: No such file or directory


In [24]:
pop = pd.read_hdf('pop.hdf')

In [28]:
phot = pd.HDFStore('phot.hdf')

In [26]:
pop

,f,max,min,t0
idx,,,,
0,0.002,0.5,-0.2,59580.03
1,0.004,1.0,-0.5,59580.05


In [29]:
phot.get('0').query('objID == 0')

,mjd,fluxerr,deviations,flux,fieldID,zpsys,band,fiveSigmaDepth,obsHistID,objID,ModelFlux,zp
expID,,,,,,,,,,,,
0_1,59580.033829,8.527999e-07,0.066517,0.941513,316,ab,lssty,21.021236,1,0,0.941513,0.0
0_14,59580.039637,7.464606e-10,0.302472,-0.994376,1118,ab,lssty,21.067249,14,0,-0.994376,0.0
0_13,59580.039190,7.301663e-10,-0.634322,-0.993118,1220,ab,lssty,21.091149,13,0,-0.993118,0.0
0_12,59580.038741,7.055271e-10,-0.362741,-0.942123,1212,ab,lssty,21.128319,12,0,-0.942123,0.0
0_11,59580.038293,7.217953e-10,-0.672460,-0.844112,1110,ab,lssty,21.103635,11,0,-0.844112,0.0
0_10,59580.037847,7.537305e-10,-0.359553,-0.704634,1021,ab,lssty,21.056753,10,0,-0.704634,0.0
0_9,59580.037401,7.814629e-10,-0.813146,-0.530260,957,ab,lssty,21.017621,9,0,-0.530260,0.0
0_8,59580.036955,7.766471e-10,-1.726283,-0.329626,857,ab,lssty,21.024316,8,0,-0.329626,0.0
0_7,59580.036509,7.732405e-10,0.177426,-0.112668,770,ab,lssty,21.029077,7,0,-0.112668,0.0
